<a href="https://colab.research.google.com/github/JazminCaruso/AyP-Python/blob/main/Jazm%C3%ADn_Caruso_Rojo_Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo RSA y tests de primalidad

Al finalizar este proyecto deberás haber escrito  los algoritmos necesarios para implementar el algoritmo RSA en la "vida diaria". Es decir,  se podrían usar los algoritmos para implementar un sistema completamente funcional. 

Para implementar el algoritmo RSA son necesarios dos componentes importantes. El primero es que debemos saber hacer eficientemente exponenciación modular con exponentes "grandes". El segundo componente importante es que debemos poder determinar en forma eficiente si un número, también, "grande",  es primo o no. Esto nos permitirá encontrar números primos para implementar RSA. 

A lo largo del proyecto se darán las herramientas para poder desarrollar estas dos componentes y finalmente, lo más sencillo, implementar el algoritmo RSA. 



## Método binario para exponenciacion modular

Ver: [https://es.wikipedia.org/wiki/Exponenciaci%C3%B3n_modular](https://es.wikipedia.org/wiki/Exponenciaci%C3%B3n_modular)

Sean `a, d, n` enteros positivos se desea calcular `a**d % n`
1.  Se calcula la expresión binaria de $d$. 
Si $d$ en base $2$ es $d_m...d_0$, entonces
$$d = \sum_{i=0}^{m} d_i \cdot  2^i$$
Entonces
\begin{align*}
a^d &= \prod_{i=0}^{m} a^{d_i \cdot 2^i} \\ 
             &= \prod_{i=0}^{m} (a^{2^i})^{d_i}.
\end{align*} 
Usando lo anterior podemos calcular `c = a**d % n` recursivamente:
        c = (a**(2**0))**d_0 % n        (caso base)
        c = c * (a**(2**i))**d_i % n    (paso i)
2. El cálculo anterior no produce gran beneficio en la eficiencia de la exponenciación modular, pues `a**(2**i) % n` sigue siendo costoso de calcular. Sin embargo, si $a^2 \equiv a_0 \pmod{n}$, obtenemos la igualdad
$$
a^{2^i} = a^{2\cdot 2^{i-1}} = (a^{2})^{2^{i-1}} \equiv a_0^{2^{i-1}} \pmod{n}.
$$ 
Luego,  en $i$ pasos podemos obtener el valor `a**(2**i) % n`.

3. Los dos pasos anteriores dan un algoritmo eficiente para calcular `c = a**d % n`, sin embargo, utilizando la idea de escribir el exponente en base 2, se puede obtener un algoritmo eficiente y más simple en su implementación. La idea es la siguiente:
$$   d = d_0 \cdot 2 + r  \quad  \Rightarrow \quad  a^d \equiv (a^2)^{d_0}  a^r \pmod n \tag{1}.$$
Si $a^2 \equiv a_0 \pmod n$, obtenemos
$$   d = d_0 \cdot 2 + r  \quad  \Rightarrow \quad  a^d \equiv {a_0}^{d_0}  a^r \pmod n \tag{2}.$$ 
Supongamos que queremos calcular $r$ tal que 
$$3^{9} \equiv r\; (\operatorname{mod} 5),$$
con $0 \le r < 5$. Esto no es más que calcular
         3**9 % 5.
En  este caso, podemos hacer este cálculo en Python directamente, pero mostraremos como es el método para ejemplificar. Lo  que haremos es lo siguiente: como `9 = 2*4 + 1`, tenemos por la fórmula (2) que 
        3**9 % 5 = (3**2)**4 * 3 % 5
                 =      4**4 * 3 % 5     # 3**2 % 5 = 9 % 5 = 4
                 = (4**2)**2 * 3 % 5
                 =      1**2 * 3 % 5     # 4**2 % 5 = 16 % 5 = 1
                 = 3 % 5
                 = 3
Supongamos ahora que queremos calcular 
         5**1125899986842625 % 100000037.
Hacer este cálculo en Python directamente no nos da un resultado satisfactorio. Haga el intento para  convencerse. Pero  como `1125899986842625 < 2**51`, usando el método de las fórmulas (1) y (2) necesitaremos alrededor de 50 pasos del tipo 
        c = x**2 * 5**r * c % 100000037
donde `0 <= x, c <= 100000037` y `r = 0, 1`. La clave es que cada uno de estos cálculos se realiza con facilidad en Python. 

**Ejercicio 1.** Sean `a, d, n` enteros positivos definir la función

        pot_mod(a, d, n) 

que devuelve `a**d % n` (`a` a la `d` módulo `n`) usando el  método binario de exponenciación modular.

In [ ]:
def pot_mod(a: int, d: int, n: int) -> int:
    # pre: a, d, n enteros positivos
    # post: devuelve a**d % n calculado por el método binario de exponenciacion modular
    c = 1
    # completar
    return c


In [ ]:
# Tests (los resultados deberían ser casi instantaneos)
print(pot_mod(2,4,15)) # 1
print(pot_mod(7, 385, 11)) # 10
print(pot_mod(5,1125899986842625, 100000037 )) # 98770120


## Test de primalidad de  Miller-Rabin 

El test de primalidad Miller-Rabin es una prueba probabilística de primalidad: un algoritmo que determina si un número dado es probable que sea primo.

Este test sigue siendo ampliamente utilizado en la práctica (en RSA, por ejemplo) y es una de las pruebas más simples y rápidas conocidas.

Dado $m$ un entero positivo.
- Si le hacemos el test a $m$ y no supera la prueba, entonces el número no es primo. 
- Si hacemos $k$ veces el test y $m$ supera las $k$ pruebas, entonces  $m$ tiene la probabilidad $1 - ( 1 / 4^k)$ de ser primo. 

Ver: [https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test)

&nbsp;

**Números probables primos**

Sea $n > 2$ un entero impar, entonces $n = 2^s \cdot d + 1$ con $d$ impar. Sea $a$ entero  tal que $0 < a < n$. Entonces diremos que $n$ es _fuertemente probable primo (FPP) respecto a la base_ $a$ si se cumple  
- $a^{d} \equiv 1 \pmod{n}$, o
- $a^{2^r\cdot\, d} \equiv -1 \pmod{n}$  para algún $r$ tal que $0 \le r < s$.

&nbsp;

Con la aplicación del teorema de Fermat y la ecuación lineal de congruencia se prueba que todo número primo es FPP respecto a cualquier base. El contrarrecíproco de esta afimación nos dice que un número que no es FPP respecto a alguna base es compuesto.

Ahora bien, un número $n$ que es FPP respecto a todas las bases $ 0 < a < n$ es primo, pero este cáclulo es computacionalmente imposible para primos grandes.  

Por otro lado, un número $n$ que es FPP respecto alguna base $ 0 < a < n$ podría ser compuesto, pero hay una probabilidad mayor que 0.75 de que sea primo. La verificación con diferentes bases de que un número es FPP acerca a 1 la probabilidad de que el número sea primo.  


**Algoritmo**

El test  de primalidad de Miller-Rabin se basa en las observaciones realizadas más arriba: sea `n` entero positivo impar entonces, `n = 2**s * d + 1` con `d` impar, y sea `k` entero positivo.  
1. Elegir al  azar `a` entero tal que `0 < a < n`.
2. Verificar que  `n` es FPP respecto a la base `a`.
3. Repetir 1. y 2. `k` veces.

Si `n` es FPP las `k` veces,  entonces decimos que `n` supera el test de primalidad de Miller-Rabin (y lo consideramos primo).  

**Ejercicio 2**

1. Escribir la función `fpp()` que determina  si un número `n` es fuertemente probablemente primo respecto a una base `a`.
2. Escribir el test de primalidad de Miller-Rabin `test_Miller_Rabin()`.  

_Observación._ Para la definición de `test_Miller_Rabin()` puede usarse  la función `fpp()` `k`-veces aunque esto no sería muy conveniente (¿por qué?). Si lo desea, puede reformular la modularización con funciones auxiliares (lo más elegante) o simplemente reescribir parte del código de `fpp()` dentro de `test_Miller_Rabin()`.

In [ ]:
import random

def pot2(n: int):
    # pre: n > 0, n impar
    # post: devuelve s, d tal n = 2**s * d + 1,  con d impar.
    pass # completar


def fpp(n, a: int) -> bool:
    # pre:  n impar, 0 < a < n
    # post: devuelve True si n = es FPP respecto a a. False en caso contrario
    ret = False
    pass # Completar
    return ret

def test_Miller_Rabin(n: int, k: int) -> bool:
    # pre: n > 2, n impar, k > 0
    # post: si n es FPP k-veces (con base al azar) devuelve True. En  caso  contrario  devuelve False. 
    ret = False
    pass # Completar
    return ret

In [ ]:
# Tests: si las funciones están bien implementadas los siguiente debería ser muy rápido. 

print(fpp(17, 5)) # True
print(fpp(3221225473, 53)) # True

primo = True
for i in range(27):
  primo = primo and fpp(27, i)
print(primo) # False

print(test_Miller_Rabin(31, 4)) # True
print(test_Miller_Rabin(351, 10)) # probablemente False
print(test_Miller_Rabin(10**8+37, 5)) # True
print(test_Miller_Rabin(2074722246773485207821695222107608587480996474721117292752992589912196684750549658310084416732550077, 100)) # True
print(test_Miller_Rabin(323000000000023902000000000442187, 100)) # False (probablemente)
print(test_Miller_Rabin(2**500 + 3, 50)) # False (probablemente)

**Ejercicio 3**

Los primos _grandes_ son muy importantes para el uso en criptografía de clave pública, en particular para el algoritmo RSA. 

El objetivo de este ejercicios es calcular de manera eficiente números primos aleatorios muy grandes con un tamaño de bit específico. 

El método estándar para implementar un generador de números primos aleatorios se da a continuación:

1. Preseleccione un número aleatorio con el tamaño de bits deseado 
2.Asegúrese de que el número elegido no sea divisible por los primeros cientos de números primos (estos están pregenerados)
3. Aplique un cierto número de iteraciones de la prueba de primalidad de Rabin-Miller. Un número aceptable son 50 repeticiones. 

- Implementar estos pasos.
- Definir una función que encuentre un número de 100 dígitos que supere Miller-Rabín 50 veces. 


In [ ]:
# Usar la siguiente función para encontrar primos de 2 a n. 
def criba_w(n): # de Wikipedia en inglés.
    # pre: n  entero positivo
    # post: devuleve la lista de los primos <= n
    a = [True]*(n+1) # Hace un  lista de n + 1 elementos cada uno True [True, True, ..., True]
    for i in range(2, int(n**0.5) + 1): # por observación 1
        if a[i] == True:
            for j in range(i**2, n+1, i ): # por observación 2
                a[j] = False
    # Si a[i] == True,  entonces i es primo (i >= 2)
    return [i for i in range(2,n+1) if a[i] == True]
  


In [ ]:
# Dado t, completar con las funciones que permitan encontrar un número al azar de longitud t que no sea divisible por los primos <= n (para n dado)

## El criptosistema RSA

Una de las aplicaciones más elementales y difundidas de la aritmética es en el diseño de sistemas criptográficos. El RSA es el más conocido de ellos y lo implementaremos a continuación.

El RSA es un sistema de clave pública, lo que en este caso significa que  el receptor conoce una clave privada $y$ (no compartida por nadie) y publicita una clave pública $x$. Si alguien desea enviar un mensaje $M$ debe hacer $M'=f(M,x)$, lo envía al receptor que para decodificarlo debe hacer $M=g(M',y)$, donde $f$ y $g$ son funciones cuidadosamente elegidas. $M'$ puede ser enviado en forma no segura, pues el único que puede decodificarlo es el poseedor de la clave privada. También $f$ y $g$ son públicas, los únicos secretos son $M$ e $y$.

Una ventaja evidente de los sistemas de clave pública es que no es necesario poner en conocimiento del emisor ninguna clave confidencial, más aún cualquier persona puede enviar en forma confidencial datos a otra persona que ha publicitado su clave.



**Idea del  algoritmo**

Supongamos que la persona $B$  quiere enviar a la persona $A$ un mensaje $m$ pero encriptado de tal forma que sólo $A$ pueda leer su contenido. Por su parte $A$ hace públicos dos números $e$ y $n$ que son los que se utilizarán para encriptar los mensajes que le envíen. 

Entonces a partir de $m$ la persona $B$ genera un mensaje cifrado $c$ mediante la siguiente operación:
$$
    c\equiv m^e\ \pmod{n}\ ,
$$
donde $e$ y $n$ es la clave pública de $A$.

Ahora $A$ recupera le mensaje $m$ a partir del mensaje en clave $c$ mediante la operación inversa dada por
$$
    m\equiv c^d\ \pmod{n}\ ,
$$
donde $d$ es la clave privada que solo $A$ conoce.

**Elección de claves**


Dados primos distintos $p$ y $q$ suficientemente grandes. 

- La _clave pública_ es $(n, e)$ con  
    - $n = pq$,   
    - $1 < e < (p-1)(q-1)$ tal que $$\operatorname{mcd}(e, (p-1)(q-1)) = 1.$$ 
- La _clave privada_ es un $d$ tal que 
$$ed \equiv 1 \pmod{(p-1)(q-1)} \quad \wedge \quad 0 \le d <(p-1)(q-1)$$



_Observacion_. Algunos comentarios sobre la elección de $p,q,e,d$.
- Los dos primos $p$ y $q$ deberían tener alrededor de $300$ dígitos cada uno (longitud considerada segura en este momento).
- El número $e$ puede elegirse pequeño y se selecciona haciendo prueba y error con el algoritmo de Euclides, es decir probando hasta encontrar un $e$ tal que $\operatorname{mcd}(e, (p-1)(q-1)) = 1$.
- La existencia de $d$ está garantizada por la ecuación lineal de congruencia, pues $e$ y $(p-1)(q-1)$ son coprimos.

**Ejercicio 4.** dados `p` y `q` dos números primos, definir la función

        clave_pub(p, q)
que devuelve  `(n, e)` tal que `n == p*q` y `mcd(e, (p-1)*(q-1)) == 1`. 

Definir también la función
        
        clave_priv(p, q) 
que devuelve un entero `d` tal que `e * d % (p-1)*(q-1) == 1`.

In [ ]:
import math

def clave_pub(p,q: int) -> tuple([int, int]):
    # pre: p, q números primos
    # post: devuelve (n, e) tal que n == p*q y mcd(e, (p-1)*(q-1)) == 1 
    pass # completar
    return 0, 0
    

# Se debe usar la siguiente función (o similar) para resolver ed = 1 (mod (p-1)(q-1))

def mcd_extendido(a, b: int) -> tuple([int, int, int]):
  # pre: a y b son números positivos
  # post: devuelve d, s, t tal que d = mcd(a,b) = a*s + b*t
  pass # completar
  return 0, 0, 0


La  función `mcd_extendido()` nos permite calcular la clave privada: sean `p, q`números primos y `e` tal que `mcd(e, m) = 1`,  donde `m = (p-1)*(q-1)`. Si  `(1, s, t) = mcd_extendido(e, m)`, luego `1 = s*e + t*m` lo cual implica `s*e % m = 1`. Es decir la clave privada es `s`. 

In [ ]:
def clave_priv(p, q, e: int) -> int:
    # pre: p, q números primos, mcd(e, (p-1)*(q-1)) == 1
    # post: devuelve d tal que e * d % (p-1)*(q-1) == 1
    pass # Completar
    return 0


In [ ]:
# Tests
p, q = 31, 47
(n, e) = clave_pub(p, q)
print('Clave pública:',n,e) # 1457, 7
print(n == p*q and math.gcd(e, (p-1)*(q-1)) == 1) # True
d = clave_priv(p, q, e)
print('Clave privada:',d) # 1183
print((e * d) % ((p-1)*(q-1)) == 1) # True

p, q = 347, 120413
(n, e) = clave_pub(p, q)
print('Clave pública:',n,e) # 41783311 3
print(n == p*q and math.gcd(e, (p-1)*(q-1)) == 1) # True
d = clave_priv(p, q, e)
print('Clave privada:',d) # 27775035
print((e * d) % ((p-1)*(q-1)) == 1) # True

### Encriptar y desencriptar mensajes

El receptor de mensajes publicita la clave pública $(n, e)$. Obviamente no da a conocer ni $p$, ni $q$ y mantiene segura la clave privada $d$. Como mencionamos anteriormente, el envío del mensaje y su decodificación requiere dos pasos
1. El  emisor desea encriptar un número $m \in \{0,\ldots,n-1\}$ y para ello calcula $$c \equiv m^e \pmod{n}$$ y  envía $c$ al receptor.
2. El receptor desea desencriptar el mensaje, es decir usando la clave pública $(n, e)$ y el entero $c$, desea recuperar el mensaje original y  calcula $$c^d \pmod{n}.$$.  
Se puede demostrar que $m \equiv c^d \pmod{n}$,  es decir que se recupera el mensaje original. 

**Ejercicio 5.** dados `p` y `q` dos números primos, una clave pública `(n, e)` y una clave privada `d`, definir la función

        encriptar(m, n, e)
que encripta el número `m` tal que `1 <= m <= n` según el procedimiento explicado más arriba. 

Definir también la función
        
        desencriptar(c, d) 
que recupera el mensaje original según el procedimientoo explicado más arriba.

In [ ]:


def encriptar(m, n, e: int) -> int:
    # pre: n y e deben ser una clave pública RSA
    # post: devuelve m**e % n 
    return 0 # Cambiar

def desencriptar(n, c, d: int)  -> int:
    # pre: n es la primera clave pública, d es la clave privada, c es un mensaje encriptado. 
    # post: devuelvec**d % n
    return 0 # Cambiar

In [ ]:
# Tests

# Test 1
p, q = 23, 19
(n, e) = clave_pub(p, q)
print('Clave pública:', n, e)
d = clave_priv(p, q, e)
print('Clave privada:',d)
m = 321
# m = 425
# m = 17 
print('mensaje O:', m)
c = encriptar(m, n, e)
print('encriptado:', c)
print('mensaje D:',desencriptar(n, c, d)) # debe dar m# debe dar 321

# Test 2
p, q = 31, 47
(n, e) = clave_pub(p, q)
print('Clave pública:', n, e)
d = clave_priv(p, q, e)
print('Clave privada:',d)
m = 1321
# m = 425
# m = 17 
print('mensaje O:', m)
c = encriptar(m, n, e)
print('encriptado:', c)
print('mensaje D:',desencriptar(n, c, d)) # debe dar m

# Test 3
p, q = 347, 120413
(n, e) = clave_pub(p, q)
print('Clave pública:', n, e)
d = clave_priv(p, q, e)
print('Clave privada:',d)
m = 1321
# m = 425
# m = 17 
print('mensaje O:', m)
c = encriptar(m, n, e)
print('encriptado:', c)
# El último paso no se puede hacer sin una mejor implementación de c**d % n
print('mensaje D:',desencriptar(n, c, d)) # debe dar m




In [ ]:
# Test 4
p = 93499781867718509625243886866950083035386961920954470625316868368084084635737751120749829224678180847
q = 94712647889447080631418693313649433127300704721890445625257438462616738681520552503342426572061648387
(n, e) = clave_pub(p, q)
print('Clave pública:', n, e)
d = clave_priv(p, q, e)
print('Clave privada:',d)
m = random.randint(10**100, 10**101)
# m = 425
# m = 17 
print('mensaje O:', m)
c = encriptar(m, n, e)
print('encriptado:', c)
# El último paso no se puede hacer sin una mejor implementación de c**d % n
print('mensaje D:',desencriptar(n, c, d)) # debe dar m